NOTE: Running the whole notebook may cause errors due to excessive memory use by the visualizations. 

If this happens, try clearing cell outputs, restarting the kernel and/or even reopening the notebook as necessary.

In [ ]:
import numpy as np

import pandas as pd

import xarray as xr

import glidertools as gt
from cmocean import cm as cmo
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import altair as alt

In [ ]:
koskelo_timeseries = xr.open_dataset("../Example Glider Data/TVAR20232.nc")
koskelo_timeseries

In [ ]:
np.nanmin(koskelo_timeseries['chlorophyll'].data)

In [ ]:
np.ptp(koskelo_timeseries['chlorophyll'].data)

In [ ]:
koskelo_timeseries.to_dataframe

In [ ]:
koskelo_timeseries_df = koskelo_timeseries.to_dataframe().reset_index()
koskelo_timeseries_df

In [ ]:
koskelo_timeseries.data_vars

In [ ]:
list(koskelo_timeseries.keys())

In [ ]:
list(koskelo_timeseries.coords)

In [ ]:
np.min(koskelo_timeseries)

In [ ]:
np.max(koskelo_timeseries)

In [ ]:
np.max(koskelo_timeseries.where(np.min(koskelo_timeseries) == 0))

In [ ]:
#koskelo_timeseries_df = koskelo_timeseries.to_dataframe().reset_index()
source = pd.melt(koskelo_timeseries_df, 'time')

step = 20
overlap = 1

alt.data_transformers.enable("vegafusion")

num_of_data_vars = len(list(koskelo_timeseries.keys()))
num_of_coords = len(list(koskelo_timeseries.coords))
num_of_plots = num_of_data_vars + num_of_coords - 1

selector = alt.selection_point(fields=['variable'])

color = alt.condition(
    selector,
    alt.value("#E48A3F"),
    alt.value('lightgray')
)

base = alt.Chart(source, height=step).properties(
    #width=250,
    #height=250
).add_params(selector)

all = base.mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    #stroke='lightgrey',
    strokeWidth=0.5,
).encode(
    alt.X('time:T', timeUnit='hoursminutes')
        .title('Time'),
    alt.Y('average(value):Q')
        .axis(None),
    color=color
    #color=alt.condition(
    #    selector,
    #    alt.value("#E48A3F"),
    #    alt.value("#BFBEBF"))
).facet(
    row=alt.Row('variable:N')
        .title(None)
        .header(labelAngle=0, labelAlign='left')
).properties(
    title='Koskelo TVAR20232 Measurements',
    bounds='flush'
).resolve_scale(
    y='independent'
)

interactive = base.mark_area().encode(
    alt.X('average(value):Q').title('Avg per minute'),
    alt.Y('time:T', timeUnit='hoursminutes', scale=alt.Scale(reverse=True))
        .title('Time'),
    color=alt.Color('id:O').legend(None)
).transform_filter(
    selector
).properties(
    height=num_of_plots*step,
    width=100
).interactive()

chart = (all | interactive).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

chart

In [ ]:
# chart.save('netcdf_plot_example.html')

In [ ]:
#koskelo_timeseries_df = koskelo_timeseries.to_dataframe().reset_index()
#source = pd.melt(koskelo_timeseries_df, 'time')

# alt.data_transformers.enable("vegafusion")

num_of_data_vars = len(list(koskelo_timeseries.keys()))
num_of_coords = len(list(koskelo_timeseries.coords))
num_of_plots = num_of_data_vars + num_of_coords - 1

selector = alt.selection_point(fields=['variable'])

color = alt.condition(
    selector,
    alt.value("#E48A3F"),
    alt.value('lightgray')
)

base = alt.Chart(source, height=step).properties(
    #width=250,
    #height=250
).add_params(selector)

all = base.mark_line(
    interpolate='monotone',
    fillOpacity=0.8,
    #stroke='lightgrey',
    strokeWidth=0.5,
).encode(
    alt.X('time:T', timeUnit='hoursminutes')
        .title('Time'),
    alt.Y('average(value):Q')
        .axis(None),
    color=color
    #color=alt.condition(
    #    selector,
    #    alt.value("#E48A3F"),
    #    alt.value("#BFBEBF"))
).facet(
    row=alt.Row('variable:N')
        .title(None)
        .header(None)
        #.header(labelAngle=0, labelAlign='left')
).properties(
    title='Koskelo TVAR20232 Measurements',
    bounds='flush'
).resolve_scale(
    y='independent'
)

interactive = base.mark_line().encode(
    alt.X('average(value):Q').title('Avg per minute'),
    alt.Y('time:T', timeUnit='hoursminutes', scale=alt.Scale(reverse=True))
        .title('Time'),
    color=alt.Color('id:O').legend(None)
).transform_filter(
    selector
).properties(
    height=num_of_plots*step,
    width=100
).interactive()

legend = base.mark_point().encode(
    alt.Y('variable:N').axis(orient='right').title(None),
    color=color
).properties(
    height=num_of_plots*step,
    #width=30
).interactive()

(legend | all | interactive).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

In [ ]:
pd.melt(koskelo_timeseries_df, 'time')

In [ ]:
source = pd.melt(koskelo_timeseries_df, ['time', 'profile_index'])
source

In [ ]:
input_dropdown = alt.binding_select(options=list(range(0, 26)), name='Profile')
selection = alt.selection_point(fields=['profile_index'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('profile_index:N').legend(None),
    alt.value('lightgray')
)
opacity = alt.condition(
    selection,
    alt.value(1),
    alt.value(0.1)
)

alt.Chart(koskelo_timeseries_df).mark_point().encode(
    x='time:T',
    y='depth:Q',
    color=color,
    opacity=opacity
).properties(
    width=1200,
    height=700
).add_params(
    selection
).interactive()

In [ ]:
np.nanmax(koskelo_timeseries_df['profile_index'])

In [ ]:
[np.nanmin(koskelo_timeseries_df['depth']), np.nanmax(koskelo_timeseries_df['depth'])]

In [ ]:
dropdown = alt.binding_select(
    options=koskelo_timeseries_df.columns.to_list()[1:-2],
    name='Y-axis column '
)
ycol_param = alt.param(
    value=koskelo_timeseries_df.columns.to_list()[1],
    bind=dropdown
)

legend_selection = alt.selection_point(fields=['profile_index'], bind='legend')

color = alt.condition(
    legend_selection,
    alt.Color('profile_index:N'),
    alt.value('lightgray')
)
opacity = alt.condition(
    legend_selection,
    alt.value(1),
    alt.value(0.1)
)

alt.Chart(koskelo_timeseries_df).mark_point().encode(
    x='time:T',
    y=alt.Y('y:Q').title(''),
    color=color,
    opacity=opacity
).transform_calculate(
    y=f'datum[{ycol_param.name}]'
).properties(
    width=1300,
    height=700
).add_params(
    ycol_param,
    legend_selection
).interactive()

In [ ]:
interval = alt.selection_interval()

dropdown = alt.binding_select(
    options=koskelo_timeseries_df.columns.to_list()[1:-2],
    name='Y-axis column '
)
ycol_param = alt.param(
    value=koskelo_timeseries_df.columns.to_list()[1],
    bind=dropdown
)

legend_selection = alt.selection_point(fields=['profile_index'], bind='legend')

color = alt.condition(
    legend_selection,
    alt.Color('profile_index:N'),
    alt.value('lightgray')
)
opacity = alt.condition(
    legend_selection,
    alt.value(1),
    alt.value(0.1)
)

base = alt.Chart(koskelo_timeseries_df).mark_point().encode(
    x='time:T',
    y=alt.Y('y:Q').title(''),
    color=color,
    opacity=opacity
).add_params(
    ycol_param,
    legend_selection,
    interval
).transform_calculate(
    y=f'datum[{ycol_param.name}]'
)

chart = base.encode(
    x=alt.X('time:T'),
    y=alt.Y('y:Q').title('')
).properties(
    width=1300,
    height=300
)

view = base.properties(
    width=1300,
    height=300,
).transform_filter(
    interval
)

chart & view

In [ ]:
koskelo_timeseries_df.columns.to_list()[1:-2]

In [ ]:
koskelo_timeseries_df.columns.to_list()[1]

In [ ]:
pyglider_timeseries = xr.open_dataset("../Example Glider Data/TVAR20232.nc")
pyglider_timeseries["profile_num"] = pyglider_timeseries["profile_index"]
tvar_data = gt.load.voto_seaexplorer_dataset(pyglider_timeseries)
tvar_data = tvar_data.drop_vars(["profile_num"])
tvar_data

In [ ]:
dives = tvar_data.dives
depth = tvar_data.depth
salt = tvar_data.salinity

x = np.array(dives)  # ensures these are arrays
y = np.array(depth)

gt.plot(dives, depth, salt, cmap=cmo.haline, robust=True)
# plt.xlim(50, 150)
plt.title('Original Data')
plt.show()

In [ ]:
tvar_data

gt.plot.section3D(
    tvar_data.dives, tvar_data.depth, tvar_data.longitude, tvar_data.latitude, tvar_data.salinity,
    zmin=-50, vmax=.999, vmin=.005
)

In [ ]:
def calc_physics(
    variable,
    dives,
    depth,
    spike_window=3,
    spike_method="minmax",
    iqr=1.5,
    z_score=None,
    depth_threshold=400,
    mask_frac=0.2,
    savitzky_golay_window=11,
    savitzky_golay_order=2,
    verbose=True,
    name="Physics Variable",
):
    """
    A standard setup for processing physics variables (temperature, salinity).

    The function applies a neighbourhood interquartile range (IQR)
    outlier filter, the Briggs et al. (2011) spike filter
    followed by a Savitzky-Golay smoothing function.

    The Savitzky-Golay filter is demonstrated well on wikipedia:
    https://en.wikipedia.org/wiki/Savitzky-Golay_filter
    """

    from numpy import array, isnan

    from glidertools.helpers import printv

    from inspect import currentframe as getframe

    from glidertools.cleaning import (
        despike,
        horizontal_diff_outliers,
        outlier_bounds_iqr,
        savitzky_golay,
    )

    # an interpolation step is added so that no nans are created.
    # Note that this interpolates on the flattened series
    # var = variable.copy()  # attribute preservation

    # If horizontal_diff_outliers() z score threshold not given, use one based on
    # outlier_bounds_iqr() multiplier
    z_score = 0.67*(1+2*iqr) if z_score is None else z_score

    x = array(dives)
    y = array(depth)
    # z = array(variable)
    z = variable.copy()
    printv(verbose, "\n" + "=" * 50 + "\n{}:".format(name))

    if iqr:
        nans_before = isnan(array(z)).sum()
        z = outlier_bounds_iqr(z, multiplier=iqr)
        nans_after = isnan(array(z)).sum()
        n_masked = nans_after - nans_before
        printv(
            verbose,
            "\tRemoving outliers with IQR * {}: {} obs".format(iqr, n_masked),
        )

    if spike_window:
        z = despike(z, spike_window, spike_method)[0]
        printv(
            verbose,
            "\tRemoving spikes with rolling median (spike window={})".format(
                spike_window
            ),
        )

    if depth_threshold:
        z = horizontal_diff_outliers(x, y, z, z_score, depth_threshold, mask_frac)
        printv(
            verbose,
            ("\tRemoving horizontal outliers " "(fraction={}, multiplier={})").format(
                mask_frac, z_score
            ),
        )

    if savitzky_golay_window:
        printv(
            verbose,
            ("\tSmoothing with Savitzky-Golay filter " "(window={}, order={})").format(
                savitzky_golay_window, savitzky_golay_order
            ),
        )
        z = savitzky_golay(z, savitzky_golay_window, savitzky_golay_order)

    # z = gt.transfer_nc_attrs(getframe(), var, z, "_processed")
    z.name = variable.name + "_processed"
    z.attrs["comment"] = "smoothed data with outliers removed"

    return z

In [ ]:
dives = tvar_data.dives
depth = tvar_data.depth
salt = tvar_data.salinity

x = np.array(dives)  # ensures these are arrays
y = np.array(depth)

salt_qc = calc_physics(salt, x, y, depth_threshold=400, spike_method='median')

# PLOTTING
fig, ax = plt.subplots(3, 1, figsize=[9, 8.5], sharex=True, dpi=90)

gt.plot(x, y, salt, cmap=cmo.haline, ax=ax[0])
gt.plot(x, y, salt_qc, cmap=cmo.haline, ax=ax[1])
gt.plot(x, y, salt_qc - salt, cmap=cm.RdBu_r, vmin=-0.02, vmax=0.02, ax=ax[2])

[a.set_xlabel('') for a in ax]

ax[0].cb.set_label('Original Data')
ax[1].cb.set_label('Cleaned Data')
ax[2].cb.set_label('Difference from Original')

plt.show()

In [ ]:
salt_qc

In [ ]:
gt.plot.section3D(
    tvar_data.dives, tvar_data.depth, tvar_data.longitude, tvar_data.latitude, salt_qc,
    zmin=-50, vmax=.999, vmin=.005
)